In [1]:
%matplotlib inline
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd

camb_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0,camb_path)
import camb
from camb import model, initialpower
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))

Using CAMB 1.3.6 installed at /usr/local/anaconda/envs/py311forge/lib/python3.11/site-packages/camb


In [14]:
#Set up a new set of parameters for CAMB
pars = camb.CAMBparams()
#This function sets up CosmoMC-like settings, with one massive neutrino and helium set using BBN consistency
pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0, tau=0.06)
pars.InitPower.set_params(As=2e-9, ns=0.965, r=0)
pars.set_for_lmax(2500, lens_potential_accuracy=0);
pars.set_matter_power(redshifts=[0.,0.5,0.7,1.0], kmax=2.0)
#calculate results for these parameters
results = camb.get_results(pars)

Note: redshifts have been re-sorted (earliest first)


In [15]:
fsigma8=results.get_fsigma8()
fsigma8

array([0.42826301, 0.45891582, 0.4715776 , 0.42573843])

In [27]:
covmat_file='/Users/nguyenmn/cobaya/cobaya_cosmo/data/bao_data/sdss_DR16_BAOplus_LRG_FSBAO_DMDHfs8_covtot.txt'
covmat=np.loadtxt(covmat_file,usecols=range(9),delimiter=' ')
print(covmat[2,2],covmat[2,5],covmat[2,8],covmat[5,2],covmat[5,5],covmat[5,8],covmat[8,2],covmat[8,5],covmat[8,8])
new_covmat=np.array([[covmat[2,2],covmat[2,5],covmat[2,8]],[covmat[5,2],covmat[5,5],covmat[5,8]],[covmat[8,2],covmat[8,5],covmat[8,8]]])
print(new_covmat.shape)
print(new_covmat)
np.savetxt('./data/fsigma8_test_covmat.txt',new_covmat,delimiter=' ')

0.00203355 0.000811829 0.0 0.000811829 0.00142289 0.0 0.0 0.0 0.0019616235334774944
(3, 3)
[[0.00203355 0.00081183 0.        ]
 [0.00081183 0.00142289 0.        ]
 [0.         0.         0.00196162]]


In [3]:
data_file='~/cobaya/cobaya_cosmo/data/bao_data/sdss_DR16_BAOplus_LRG_FSBAO_DMDHfs8.dat'
data = pd.read_csv(data_file,header=None, index_col=None, sep=r"\s+", comment="#")
data.columns = ["z", "value", "observable"]
data

,z,value,observable
0,0.380,10.270000,DM_over_rs
1,0.380,24.890000,DH_over_rs
2,0.380,0.497400,f_sigma8
3,0.510,13.380000,DM_over_rs
4,0.510,22.430000,DH_over_rs
5,0.510,0.459000,f_sigma8
6,0.698,17.645955,DM_over_rs
7,0.698,19.769664,DH_over_rs
8,0.698,0.473004,f_sigma8


In [4]:
zs = {obs: data.loc[data["observable"] == obs, "z"].values
      for obs in data["observable"].unique()}

In [35]:
theory_reqs = {
             "DV_over_rs": {
                 "angular_diameter_distance": {"z": zs.get("DV_over_rs", None)},
                 "Hubble": {"z": zs.get("DV_over_rs", None)},
                 "rdrag": None},
             "rs_over_DV": {
                 "angular_diameter_distance": {"z": zs.get("rs_over_DV", None)},
                 "Hubble": {"z": zs.get("rs_over_DV", None)},
                 "rdrag": None},
             "DM_over_rs": {
                 "angular_diameter_distance": {"z": zs.get("DM_over_rs", None)},
                 "rdrag": None},
             "DA_over_rs": {
                 "angular_diameter_distance": {"z": zs.get("DA_over_rs", None)},
                 "rdrag": None},
             "DH_over_rs": {
                 "Hubble": {"z": zs.get("DH_over_rs", None)},
                 "rdrag": None},
             "Hz_rs": {
                 "Hubble": {"z": zs.get("Hz_rs", None)},
                 "rdrag": None},
             "f_sigma8": {
                 "fsigma8": {"z": zs.get("f_sigma8", None)},
             },
             "F_AP": {
                 "angular_diameter_distance": {"z": zs.get("F_AP", None)},
                 "Hubble": {"z": zs.get("F_AP", None)}}}
requisites = {}
for obs in data["observable"].unique():
    requisites.update(theory_reqs[obs])
requisites

{'angular_diameter_distance': {'z': array([0.38 , 0.51 , 0.698])},
 'rdrag': None,
 'Hubble': {'z': array([0.38 , 0.51 , 0.698])},
 'fsigma8': {'z': array([0.38 , 0.51 , 0.698])}}

In [32]:
def theory_fun(z,obs):
    if obs == "DM_over_rs":
        return z*1
    elif obs == "DH_over_rs":
        return z*2
    elif obs == "f_sigma8":
        return z*3

In [68]:
theory = np.array([[theory_fun(z,obs) for z,obs
                   in zip(data["z"],data["observable"])]]).T
print(theory.shape)

(9, 1)


In [61]:
theory = np.array([theory_fun(z,obs="f_sigma8") for z
                   in data["z"]])
theory.shape

(9,)